In [1]:
import qtlsearch
from IPython.display import Image,SVG
search = qtlsearch.SEARCH(
    "http://pbg-ld.candygene-nlesc.surf-hosted.nl:8890/sparql", 
    "http://sparql.omabrowser.org/sparql",
    "https://sparql.uniprot.org/sparql")

In [13]:
 #tomato
tg1 = "Solyc10g075170.1"
tg2 = "Solyc10g076240.1"

#tg1 = "Solyc12g056530.1"
#tg2 = "Solyc12g056530.1"

tg1 = "gene72_0-i22"
tg2 = "gene78_0-i22"

#tomato
tg1 = "Solyc10g075130.1"
tg2 = "Solyc10g076240.1"

In [14]:
intervalT = search.compute_interval(tg1, tg2)
#locationT = search.get_location("Solyc12g056530.1")
#d=100000
#intervalT = search.make_interval(
#    locationT.iloc[0]["begin_ref"], 
#    locationT.iloc[0]["begin_pos"]-d, 
#    locationT.iloc[0]["end_pos"]+d)
genesT = search.interval_genes(intervalT)

,location
gene_id,
Solyc10g075130.1,chromosome 10:58849843-58850038
Solyc10g076240.1,chromosome 10:59082990-59084119


In [23]:
qtls = [genesT.index]
go_annotations = search.get_child_annotations("GO:0045892")
#print(go_annotations)

In [21]:
import pandas as pd

p_uniprot_reviewed = 1.00
p_uniprot_unreviewed = 0.95
loss_up_ortholog = 0.85
loss_down_ortholog = 0.85
loss_up_paralog = 0.7225
loss_down_paralog = 0.7225

#go_annotations = search.get_child_annotations("GO:0010154")

hog_group_trees = pd.Series()
hog_group_genes = pd.Series()
#root in hog tree for each gene
qtl_gene_roots = pd.Series()

qtl_gene_protein = pd.Series()
gene_p_initial = pd.Series()


for qtl in qtls:
    for gene in qtl:
        if not(gene in qtl_gene_protein.keys()):
            p_qtl_initial = 1.0/len(qtl)        
            #start searching
            print("Search for "+gene)
            #first, go up in the hog-tree
            parent_groups = search.get_parent_groups(gene)
            if len(parent_groups)>0:
                #define the root of the tree
                qtl_gene_roots[gene] = parent_groups.index[0]
                qtl_gene_protein[gene] = parent_groups.loc[parent_groups.index[0]].protein
                print("- root is "+qtl_gene_roots[gene])
                if qtl_gene_roots[gene] in hog_group_trees.index:
                    print("- tree already created")
                else:    
                    #go down the hog-tree, just to find tree structure
                    hog_group_trees[qtl_gene_roots[gene]] = search.get_child_groups(qtl_gene_roots[gene])
                    hog_group_trees[qtl_gene_roots[gene]].loc[:,"p_initial"] = pd.Series(0.0, index=hog_group_trees[qtl_gene_roots[gene]].index)        
                    hog_group_trees[qtl_gene_roots[gene]].loc[:,"p_up"] = pd.Series(0.0, index=hog_group_trees[qtl_gene_roots[gene]].index)        
                    hog_group_trees[qtl_gene_roots[gene]].loc[:,"p_down"] = pd.Series(0.0, index=hog_group_trees[qtl_gene_roots[gene]].index)        
                    print("- tree of groups fetched: "+str(len(hog_group_trees[qtl_gene_roots[gene]])))
                    #go down again, now to find proteins
                    tree_proteins = search.get_child_proteins(qtl_gene_roots[gene])
                    tree_proteins_uniprot = search.get_child_proteins_uniprot(qtl_gene_roots[gene])
                    print("- proteins within tree fetched: "+str(len(tree_proteins)))
                    print("- uniprot proteins within tree fetched: "+str(len(tree_proteins_uniprot)))
                    #create final list of checked proteins
                    hog_group_genes[qtl_gene_roots[gene]] = tree_proteins
                    hog_group_genes[qtl_gene_roots[gene]].loc[:,"reviewed"] = pd.Series("unknown", index=hog_group_genes[qtl_gene_roots[gene]].index)
                    hog_group_genes[qtl_gene_roots[gene]].loc[:,"p_initial"] = pd.Series(0.0, index=hog_group_genes[qtl_gene_roots[gene]].index)
                    hog_group_genes[qtl_gene_roots[gene]].loc[:,"p_up"] = pd.Series(0.0, index=hog_group_genes[qtl_gene_roots[gene]].index)
                    hog_group_genes[qtl_gene_roots[gene]].loc[:,"p_final"] = pd.Series(0.0, index=hog_group_genes[qtl_gene_roots[gene]].index)
                    #create uniprot list for checking
                    list_proteins = list(tree_proteins_uniprot.index)
                    #divide proteins into chunks (sparql endpoint doesn't allow huge lists)
                    n=50
                    lists_proteins = [list_proteins[i * n:(i + 1) * n] for i in range((len(list_proteins) + n - 1) // n )]          
                    checked_proteins_list = []
                    #check chunks of proteins in uniprot
                    for i in range(0,len(lists_proteins)):
                        checked_proteins_sublist = search.check_uniprot_annotations(lists_proteins[i], list(go_annotations.index))
                        print("  * check proteins ("+str((i*n)+1)+"-"+str(min((i+1)*n,len(tree_proteins)))+"): "+
                              str(len(checked_proteins_sublist))+" with required annotation")
                        #collect group labels to give some indication of origin
                        mask = list(tree_proteins_uniprot.loc[checked_proteins_sublist.index].protein.unique())
                        sublist_labels = list(tree_proteins.loc[mask].label.dropna().unique())
                        if len(sublist_labels)>0:
                              print("    -> "+", ".join(sublist_labels))
                        #add checked proteins from chunk to the collective list        
                        checked_proteins_list.append(checked_proteins_sublist)
                    checked_proteins = pd.concat(checked_proteins_list)                 
                    #update reviewed
                    hog_group_genes[qtl_gene_roots[gene]].reviewed = None
                    if qtl_gene_protein[gene] in gene_p_initial.index:
                        hog_group_genes[qtl_gene_roots[gene]].loc[qtl_gene_protein[gene],"p_initial"] = max(gene_p_initial[qtl_gene_protein[gene]], p_qtl_initial)
                    else:
                        hog_group_genes[qtl_gene_roots[gene]].loc[qtl_gene_protein[gene],"p_initial"] = p_qtl_initial
                    if len(checked_proteins)>0:
                        maskUnreviewed = hog_group_genes[qtl_gene_roots[gene]].loc[tree_proteins_uniprot.loc[checked_proteins.loc[checked_proteins.reviewed=="false"].index].protein.unique()]
                        maskReviewed = hog_group_genes[qtl_gene_roots[gene]].loc[tree_proteins_uniprot.loc[checked_proteins.loc[checked_proteins.reviewed=="true"].index].protein.unique()]
                        hog_group_genes[qtl_gene_roots[gene]].loc[maskUnreviewed.index,"reviewed"] = False
                        hog_group_genes[qtl_gene_roots[gene]].loc[maskReviewed.index,"reviewed"] = True
                        #update probability                        
                        for reviewedProtein in hog_group_genes[qtl_gene_roots[gene]][hog_group_genes[qtl_gene_roots[gene]].reviewed==True].index:
                            hog_group_genes[qtl_gene_roots[gene]].loc[reviewedProtein,"p_initial"] = max(hog_group_genes[qtl_gene_roots[gene]].loc[reviewedProtein,"p_initial"],p_uniprot_reviewed)
                        for unreviewedProtein in hog_group_genes[qtl_gene_roots[gene]][hog_group_genes[qtl_gene_roots[gene]].reviewed==False].index:
                            hog_group_genes[qtl_gene_roots[gene]].loc[unreviewedProtein,"p_initial"] = max(hog_group_genes[qtl_gene_roots[gene]].loc[unreviewedProtein,"p_initial"],p_uniprot_unreviewed)
                    for positiveProbablilityProtein in hog_group_genes[qtl_gene_roots[gene]].loc[hog_group_genes[qtl_gene_roots[gene]]["p_initial"]>0].index:
                        if positiveProbablilityProtein in gene_p_initial.index:
                            gene_p_initial[positiveProbablilityProtein] = max(hog_group_genes[qtl_gene_roots[gene]].loc[positiveProbablilityProtein,"p_initial"],gene_p_initial[positiveProbablilityProtein])
                        else:        
                            gene_p_initial[positiveProbablilityProtein] = hog_group_genes[qtl_gene_roots[gene]].loc[positiveProbablilityProtein,"p_initial"]
                    #report results
                    print("- checked "+str(len(list_proteins))+" uniprot proteins: "+str(len(checked_proteins))+" with required annotation")
                    #including details about reviewed status
                    if len(checked_proteins)>0:
                        reviewedList = checked_proteins.groupby("reviewed")
                        if len(reviewedList)>0:
                            for label,number in reviewedList.size().items():
                                print("  -> for "+str(number)+" of these "+str(len(list_proteins))+" items, reviewed is "+str(label))                     
                    print("- checked "+str(len(tree_proteins))+" proteins: "+str(len(hog_group_genes[qtl_gene_roots[gene]]["reviewed"].dropna()))+" linked to uniprot with required annotation")
                    if len(hog_group_genes[qtl_gene_roots[gene]])>0:
                        reviewedList = hog_group_genes[qtl_gene_roots[gene]].groupby("reviewed")
                        if len(reviewedList)>0:
                            for label,number in reviewedList.size().items():
                                print("  -> for "+str(number)+" of these "+str(len(hog_group_genes[qtl_gene_roots[gene]]))+" items reviewed is marked as "+str(label))                     

            else:
                #no hog groups, can't do anything with this...
                print("- no groups found")
                
#update initial probabilities from other genes and qtls    
for gene in qtl_gene_protein.keys():
    for protein in hog_group_genes[qtl_gene_roots[gene]].keys():
        if protein in gene_p_initial.keys():
            hog_group_genes[qtl_gene_roots[gene]].loc[protein,"p_initial"] = gene_p_initial[protein];
    

Search for Solyc10g075140.1
- no groups found
Search for Solyc10g075150.1
- root is http://omabrowser.org/ontology/oma#GROUP_125182
- tree of groups fetched: 1
- proteins within tree fetched: 2
- uniprot proteins within tree fetched: 2
  * check proteins (1-2): 0 with required annotation
- checked 2 uniprot proteins: 0 with required annotation
- checked 2 proteins: 0 linked to uniprot with required annotation
Search for Solyc10g075160.1
- root is http://omabrowser.org/ontology/oma#GROUP_432941
- tree of groups fetched: 121
- proteins within tree fetched: 197
- uniprot proteins within tree fetched: 162


HTTPError: HTTP Error 414: Request-URI Too Large

In [17]:
def get_p_up(group):
    children = hog_group_trees[qtl_gene_roots[gene]].loc[hog_group_trees[qtl_gene_roots[gene]].parent==group]
    proteins = hog_group_genes[qtl_gene_roots[gene]].loc[hog_group_genes[qtl_gene_roots[gene]].group==group]
    type = hog_group_trees[qtl_gene_roots[gene]].loc[group,"type"]
    p = hog_group_trees[qtl_gene_roots[gene]].loc[group,"p_initial"]
    for protein in proteins.index:
        if type=="ortholog":
            p +=  loss_up_ortholog * hog_group_genes[qtl_gene_roots[gene]].loc[protein,"p_initial"]
        elif type=="paralog":    
            p +=  loss_up_paralog * hog_group_genes[qtl_gene_roots[gene]].loc[protein,"p_initial"]
    for child in children.index:
        if type=="ortholog":
            p +=  loss_up_ortholog * get_p_up(child)
        elif type=="paralog":    
            p +=  loss_up_paralog * get_p_up(child)   
    hog_group_trees[qtl_gene_roots[gene]].loc[group,"p_up"] = p
    #print("Group "+group+" - "+type+": "+str(p))
    return p;

def set_p_down(group):
    children = hog_group_trees[qtl_gene_roots[gene]].loc[hog_group_trees[qtl_gene_roots[gene]].parent==group]
    proteins = hog_group_genes[qtl_gene_roots[gene]].loc[hog_group_genes[qtl_gene_roots[gene]].group==group]
    type = hog_group_trees[qtl_gene_roots[gene]].loc[group,"type"]
    p = hog_group_trees[qtl_gene_roots[gene]].loc[group,"p_up"]
    parent = hog_group_trees[qtl_gene_roots[gene]].loc[qtl_gene_roots[gene],"parent"]
    if not(parent==None):
      parent_type = hog_group_trees[qtl_gene_roots[gene]].loc[group,"type"]  
      parent_p = hog_group_trees[qtl_gene_roots[gene]].loc[parent,"p_down"]
      if parent_type=="ortholog":  
          p = max(p,loss_down_ortholog*parent_p)
      elif parent_type=="paralog":
          p = max(p,loss_down_paralog*parent_p)
    hog_group_trees[qtl_gene_roots[gene]].loc[group,"p_down"] = p
    for protein in proteins.index:  
        p_protein = hog_group_genes[qtl_gene_roots[gene]].loc[protein,"p_initial"]
        if type=="ortholog":
            p_protein = max(loss_down_ortholog*p,p_protein)
        elif type=="paralog":
            p_protein = max(loss_down_paralog*p,p_protein)    
        hog_group_genes[qtl_gene_roots[gene]].loc[protein,"p_final"] = p_protein
    for child in children.index:        
        set_p_down(child)
    
for qtl in qtls:
    for gene in qtl:        
        if gene in qtl_gene_roots.keys():
            print("Compute for "+gene)
            #reset
            hog_group_trees[qtl_gene_roots[gene]].loc[:,"p_up"] = 0.0
            hog_group_trees[qtl_gene_roots[gene]].loc[:,"p_down"] = 0.0
            hog_group_genes[qtl_gene_roots[gene]].loc[:,"p_final"] = 0.0
            #go up
            get_p_up(qtl_gene_roots[gene])
            #go down
            set_p_down(qtl_gene_roots[gene])
        else:
            print("Skip "+gene)
        
        

Skip Solyc10g075140.1
Compute for Solyc10g075150.1
Compute for Solyc10g075160.1
Compute for Solyc10g075170.1
Skip Solyc10g076170.1
Compute for Solyc10g076180.1
Compute for Solyc10g076190.1
Compute for Solyc10g076200.1
Compute for Solyc10g076210.1
Compute for Solyc10g076220.1
Compute for Solyc10g076230.1


In [18]:
#report
for i in range(0,len(qtls)):
    print("=== START QTL "+str(i+1)+" ===")
    for gene in qtls[i]:
        if gene in qtl_gene_roots.keys():
            print(gene
                  +"\t"
                  +str(hog_group_genes[qtl_gene_roots[gene]].loc[qtl_gene_protein[gene],"p_initial"])
                  +"\t"
                  +str(hog_group_genes[qtl_gene_roots[gene]].loc[qtl_gene_protein[gene],"p_final"])
                  +"\t"
                  +qtl_gene_protein[gene]
                 ) 
        else:
            print(gene)    
    print("=== END QTL "+str(i+1)+" ===")
    print()

=== START QTL 1 ===
Solyc10g075140.1
Solyc10g075150.1	0.09090909090909091	0.09090909090909091	https://omabrowser.org/oma/info/SOLLC05946
Solyc10g075160.1	0.09090909090909091	0.09090909090909091	https://omabrowser.org/oma/info/SOLLC05947
Solyc10g075170.1	0.09090909090909091	0.09090909090909091	https://omabrowser.org/oma/info/SOLLC05948
Solyc10g076170.1
Solyc10g076180.1	0.09090909090909091	0.09090909090909091	https://omabrowser.org/oma/info/SOLLC05950
Solyc10g076190.1	0.09090909090909091	0.09090909090909091	https://omabrowser.org/oma/info/SOLLC05951
Solyc10g076200.1	0.09090909090909091	0.09090909090909091	https://omabrowser.org/oma/info/SOLLC05952
Solyc10g076210.1	0.0	0.04745511363636364	https://omabrowser.org/oma/info/SOLLC05953
Solyc10g076220.1	0.0	0.04745511363636364	https://omabrowser.org/oma/info/SOLLC05954
Solyc10g076230.1	0.09090909090909091	0.09090909090909091	https://omabrowser.org/oma/info/SOLLC05955
=== END QTL 1 ===



In [19]:
qtl_gene_protein["Solyc10g076180.1"]

'https://omabrowser.org/oma/info/SOLLC05950'

In [14]:
hog_group_genes[qtl_gene_roots["Solyc12g056490.1"]]

,group,label,reviewed,p_initial,p_up,p_final
protein,,,,,,
https://omabrowser.org/oma/info/RHIOR15394,http://omabrowser.org/ontology/oma#GROUP_43447...,Fungi,None,0.0000,0.0,0.000000
https://omabrowser.org/oma/info/CREFR03759,http://omabrowser.org/ontology/oma#GROUP_43447...,Ichthyosporea,None,0.0000,0.0,0.000000
https://omabrowser.org/oma/info/AMPQE22899,http://omabrowser.org/ontology/oma#GROUP_43447...,Metazoa,None,0.0000,0.0,0.000000
https://omabrowser.org/oma/info/TRIAD07600,http://omabrowser.org/ontology/oma#GROUP_43447...,Metazoa,None,0.0000,0.0,0.000000
https://omabrowser.org/oma/info/TRIAD03443,http://omabrowser.org/ontology/oma#GROUP_43447...,Metazoa,None,0.0000,0.0,0.000000
https://omabrowser.org/oma/info/TRIAD03337,http://omabrowser.org/ontology/oma#GROUP_43447...,Metazoa,None,0.0000,0.0,0.000000
https://omabrowser.org/oma/info/LINUN25118,http://omabrowser.org/ontology/oma#GROUP_43447...,Metazoa,None,0.0000,0.0,0.000000
https://omabrowser.org/oma/info/AMPQE19500,http://omabrowser.org/ontology/oma#GROUP_43447...,Metazoa,None,0.0000,0.0,0.000000
https://omabrowser.org/oma/info/TRIAD00289,http://omabrowser.org/ontology/oma#GROUP_43447...,Metazoa,None,0.0000,0.0,0.000000


In [8]:
len(checked_proteins)

0